In [1]:
!pip install neo4j

In [2]:
!pip install ipython-cypher

In [3]:
import time

In [4]:
import pandas as pd

In [5]:
from tqdm import tqdm

In [6]:
from neo4j import GraphDatabase

In [7]:
from faker import Faker
import random

In [8]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "changeme"

In [9]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [10]:
driver

Создадим узлы и отношения между ними

Очистим базу

In [11]:
# Функция для очистки базы данных
def clear_database(driver):
    # Создание сессии
    with driver.session() as session:
        # Удаление всех узлов и отношений
        session.run("MATCH (n) DETACH DELETE n")

In [12]:
# Функция для вывода информации о узлах
def print_node_info(driver):
    try:
        # Получение информации об узлах
        with driver.session() as session:
            result = session.run("MATCH (n) RETURN n")
            print("Информация об узлах:")
            for record in result:
                node_properties = record["n"]
                print(dict(node_properties.items()))
    except Exception as e:
        print(f"Произошла ошибка: {e}")

In [ ]:
clear_database(driver)

Создадим функции для генерации узлов и отношений с заданным количеством записей

In [13]:
def generate_clients(num_records):
    fake = Faker()
        
    clients_data = []
    for _ in range(num_records):
        client_name = fake.name()
        phone = fake.phone_number()
        clients_data.append({"client_name": client_name, "phone": phone})
    return clients_data

In [14]:
def generate_clients_csv(num_records):
    fake = Faker()

    names = []
    phones = []
    # Заполняем DataFrame данными о клиентах
    for _ in tqdm(range(num_records), desc="Генерация данных"):
        client_name = fake.name()
        phone = fake.phone_number()
        names.append(client_name)
        phones.append(phone)
    
    d = {"client_name": names, "phone": phones}
    clients_df = pd.DataFrame(d)

    # Записываем DataFrame в CSV файл с отслеживанием прогресса
    with tqdm(total=len(clients_df), desc="Запись в файл") as pbar:
        clients_df.to_csv("clients.csv", index=False)
        pbar.update(len(clients_df))


In [15]:
def generate_officiants(num_records):
    fake = Faker()
    
    officiants_data = []
    for _ in range(num_records):
        officiant_name = fake.name()
        officiants_data.append({"officiant_name": officiant_name})
    return officiants_data

In [16]:
def generate_tables(num_records):
    tables_data = []
    for i in range(num_records):
        table_number = i + 1 
        tables_data.append({"table_number": table_number})
    return tables_data

In [17]:
def load_clients_into_neo4j(driver):
    query_string = '''
    USING PERIODIC COMMIT 1000
    LOAD CSV WITH HEADERS FROM 'file:///clients.csv' AS row
    CREATE (c:Client {client_name: row.client_name, phone: row.phone})
    '''
    with driver.session() as session:
        session.run(query_string)

In [18]:
def create_clients_relationships(driver):
    query_string = '''
    USING PERIODIC COMMIT 1000
    LOAD CSV WITH HEADERS FROM 'file:///clients.csv' AS row
    WITH row ORDER BY rand() LIMIT 1000
    MATCH (t:Table)
    WITH row, t ORDER BY rand() LIMIT 1000
    CREATE (c:Client {client_name: row.client_name, phone: row.phone})-[:reserved]->(t)
    '''
    with driver.session() as session:
        session.run(query_string)

In [19]:
def create_nodes_and_relationships(driver, officiants_data, tables_data):
    try:
        with driver.session() as session:
            session.run("UNWIND $officiants_data AS data "
                        "CREATE (:Officiant {officiant_name: data.officiant_name})", officiants_data=officiants_data)
        with driver.session() as session:
            session.run("UNWIND $tables_data AS data "
                        "CREATE (:Table {table_number: data.table_number})", tables_data=tables_data)

        num_officiants = len(officiants_data)
        num_tables = len(tables_data)

        with driver.session() as session:
            session.run("MATCH (o:Officiant), (t:Table) "
                        "WHERE id(o) % $num_officiants = id(t) % $num_tables "
                        "CREATE (o)-[:serve]->(t)", num_officiants=num_officiants, num_tables=num_tables)    
    except Exception as e:
        print(f"Произошла ошибка: {e}")

Создадим функцию для вывода информации об узлах и отношениях

In [20]:
# Функция для вывода информации о узлах и отношениях
def print_node_and_relationship_info(driver):
    try:
        with driver.session() as session:
            # Получение информации об узлах
            print("Узлы:")
            nodes_result = session.run("MATCH (n) RETURN n")
            for record in nodes_result:
                node = record["n"]
                print("Узел:")
                for key, value in dict(node.items()).items():
                    print(f"{key}: {value}")

            # Получение информации об отношениях
            print("\nОтношения:")
            relationships_result = session.run("MATCH ()-[r]->() RETURN r")
            for record in relationships_result:
                relationship = record["r"]
                print("Отношение:")
                for key, value in dict(relationship.items()).items():
                    print(f"{key}: {value}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

In [21]:
officiants = generate_officiants(10)

In [22]:
tables = generate_tables(10)

In [23]:
start = time.time()
load_clients_into_neo4j(driver)
end = time.time() - start

In [24]:
print(f"Время заполнения таблиц: {end / 60} minutes")

Время заполнения таблиц: 0.31577502489089965 minutes


In [25]:
start = time.time()
create_nodes_and_relationships(driver, officiants, tables)
end = time.time() - start

In [26]:
print(f"Время заполнения таблиц: {end * 1000} ms")

Время заполнения таблиц: 309.53478813171387 ms


In [ ]:
# Вызов функции для вывода информации о созданных узлах и отношениях
print_node_and_relationship_info(driver)

In [28]:
create_clients_relationships(driver)

In [ ]:
# Вывод информации об узлах клиентов
with driver.session() as session:
    result = session.run("MATCH (c:Client) RETURN c")
    print("Узлы клиентов:")
    for record in result:
        print(record["c"])

Напишем запросы к данным:

In [30]:
def get_all_clients(driver):
    with driver.session() as session:
        result = session.run("MATCH (c:Client) RETURN c")
        return [record["c"] for record in result]

In [31]:
def get_all_tables(driver):
    with driver.session() as session:
        result = session.run("MATCH (t:Table) RETURN t")
        return [record["t"] for record in result]

In [32]:
def get_all_officiants(driver):
    with driver.session() as session:
        result = session.run("MATCH (o:Officiant) RETURN o")
        return [record["o"] for record in result]

In [33]:
def get_clients_reserved_for_table(driver, table_number):
    with driver.session() as session:
        result = session.run(
            "MATCH (c:Client)-[:reserved]->(t:Table) WHERE t.table_number = $table_number RETURN c",
            table_number=table_number
        )
        return [record["c"] for record in result]

In [34]:
def get_tables_served_by_officiant(driver, officiant_name):
    with driver.session() as session:
        result = session.run(
            "MATCH (o:Officiant)-[:serve]->(t:Table) WHERE o.officiant_name = $officiant_name RETURN t",
            officiant_name=officiant_name
        )
        return [record["t"] for record in result]

In [35]:
# Получение всех клиентов
start = time.time()
all_clients = get_all_clients(driver)
end = time.time() - start
print("All clients:")
for client in all_clients:
    print(client)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 52177.655935287476 ms


In [37]:
# Получение всех столов
start = time.time()
all_tables = get_all_tables(driver)
end = time.time() - start
print("All tables:")
for table in all_tables:
    print(table)

All tables:
<Node element_id='2000020' labels=frozenset({'Table'}) properties={'table_number': 1}>
<Node element_id='2000021' labels=frozenset({'Table'}) properties={'table_number': 2}>
<Node element_id='2000022' labels=frozenset({'Table'}) properties={'table_number': 3}>
<Node element_id='2000023' labels=frozenset({'Table'}) properties={'table_number': 4}>
<Node element_id='2000024' labels=frozenset({'Table'}) properties={'table_number': 5}>
<Node element_id='2000025' labels=frozenset({'Table'}) properties={'table_number': 6}>
<Node element_id='2000026' labels=frozenset({'Table'}) properties={'table_number': 7}>
<Node element_id='2000027' labels=frozenset({'Table'}) properties={'table_number': 8}>
<Node element_id='2000028' labels=frozenset({'Table'}) properties={'table_number': 9}>
<Node element_id='2000029' labels=frozenset({'Table'}) properties={'table_number': 10}>


In [38]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 12.20250129699707 ms


In [39]:
# Получение всех официантов
start = time.time()
all_officiants = get_all_officiants(driver)
end = time.time() - start
print("\nAll officiants:")
for officiant in all_officiants:
    print(officiant)


All officiants:
<Node element_id='2000000' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Julia Jones'}>
<Node element_id='2000001' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Tiffany Farrell'}>
<Node element_id='2000002' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Cody Stephens'}>
<Node element_id='2000003' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Katherine Morales'}>
<Node element_id='2000004' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Christopher Sanchez'}>
<Node element_id='2000005' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Theresa Crosby'}>
<Node element_id='2000006' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Kevin Miles'}>
<Node element_id='2000007' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Jennifer Alvarez'}>
<Node element_id='2000008' labels=frozenset({'Officiant'}) properties={'officiant_name': 'Corey Stewart'}>
<Node el

In [40]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 11.382818222045898 ms


In [41]:
# Получение всех клиентов, зарезервировавших конкретный стол
table_number = 9
start = time.time()
clients_reserved = get_clients_reserved_for_table(driver, table_number)
end = time.time() - start
print(f"\nClients reserved for table {table_number}:")
for client in clients_reserved:
    print(client)


Clients reserved for table 9:
<Node element_id='2000903' labels=frozenset({'Client'}) properties={'phone': '345.734.4227', 'client_name': 'Deanna Holt'}>
<Node element_id='2000281' labels=frozenset({'Client'}) properties={'phone': '7303384351', 'client_name': 'Melissa Williams'}>
<Node element_id='2000741' labels=frozenset({'Client'}) properties={'phone': '8017963362', 'client_name': 'Daniel Jordan'}>
<Node element_id='2000792' labels=frozenset({'Client'}) properties={'phone': '6476337904', 'client_name': 'Jason Washington'}>
<Node element_id='2000740' labels=frozenset({'Client'}) properties={'phone': '(325)849-9696x385', 'client_name': 'James Lucas'}>
<Node element_id='2000472' labels=frozenset({'Client'}) properties={'phone': '547-275-5227', 'client_name': 'Maria Schmidt'}>
<Node element_id='2000885' labels=frozenset({'Client'}) properties={'phone': '+1-875-409-1062x553', 'client_name': 'Robert Craig'}>
<Node element_id='2000791' labels=frozenset({'Client'}) properties={'phone': '66

In [42]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 146.13938331604004 ms


In [43]:
# Получение всех столов, обслуживаемых конкретным официантом
officiant_name = "Julia Jones"
start = time.time()
tables_served = get_tables_served_by_officiant(driver, officiant_name)
end = time.time() - start
print(f"\nTables served by officiant {officiant_name}:")
for table in tables_served:
    print(table)


Tables served by officiant Julia Jones:
<Node element_id='2000020' labels=frozenset({'Table'}) properties={'table_number': 1}>


In [44]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 39.96539115905762 ms


In [99]:
clear_database(driver)

In [ ]:
driver.close()